In [ ]:
!pip install http.client

In [ ]:
!pip install schedule
!pip install yfinance
!pip install kiteconnect

# Using data from Indian Stock API

In [ ]:
import http.client
import pandas as pd
import json

conn = http.client.HTTPSConnection("stock.indianapi.in")

headers = { 'X-Api-Key': "" }

conn.request("GET", "/trending", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))

df_gainers = pd.DataFrame(data['trending_stocks']['top_gainers'])
df_losers = pd.DataFrame(data['trending_stocks']['top_losers'])

df = pd.concat([df_gainers, df_losers], keys=['gainers', 'losers'])
print(df)

In [ ]:
conn.request("GET", "/fetch_52_week_high_low_data", headers=headers)

res = conn.getresponse()
data = res.read()

data = json.loads(data.decode("utf-8"))
print(data)

df_high_BSE = pd.DataFrame(data['BSE_52WeekHighLow']['high52Week'])
df_low_BSE = pd.DataFrame(data['BSE_52WeekHighLow']['low52Week'])
df_high_NSE = pd.DataFrame(data['NSE_52WeekHighLow']['high52Week'])
df_low_NSE = pd.DataFrame(data['NSE_52WeekHighLow']['low52Week'])

df_BSE = pd.concat([df_high_BSE, df_low_BSE], keys=['high52Week', 'low52Week'])
df_NSE = pd.concat([df_high_NSE, df_low_NSE], keys=['high52Week', 'low52Week'])

df = pd.concat([df_BSE, df_NSE], keys=['BSE', 'NSE'])

print(df)

# Using data from YFinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import schedule
import time

# Define stock symbol and exchange
stock_symbol = 'INFY'
exchange = 'NSE'

# Define moving average parameters
short_window = 20
long_window = 50

# Define trading parameters
buy_threshold = 0.01
sell_threshold = -0.01

# # Define Zerodha Kite API credentials (optional)
# api_key = ''
# api_secret = ''

def get_stock_data():
    stock_data = yf.download(stock_symbol + '.NS', period='1d', interval='1m')
    return stock_data

def calculate_moving_averages(stock_data):
    short_ma = stock_data['Close'].rolling(window=short_window).mean()
    long_ma = stock_data['Close'].rolling(window=long_window).mean()
    return short_ma, long_ma

def check_buy_signal(short_ma, long_ma):
    if short_ma.iloc[-1] > long_ma.iloc[-1] and short_ma.iloc[-2] <= long_ma.iloc[-2]:
        return True
    return False

def check_sell_signal(short_ma, long_ma):
    if short_ma.iloc[-1] < long_ma.iloc[-1] and short_ma.iloc[-2] >= long_ma.iloc[-2]:
        return True
    return False

def execute_trade(buy_signal, sell_signal):
    if buy_signal:
        print('Buy signal detected!')
        # Execute buy order using Zerodha Kite API (optional)
        # kite.place_order(symbol=stock_symbol, quantity=1, side='buy', type='market')
    elif sell_signal:
        print('Sell signal detected!')
        # Execute sell order using Zerodha Kite API (optional)
        # kite.place_order(symbol=stock_symbol, quantity=1, side='sell', type='market')

def main():
    stock_data = get_stock_data()
    short_ma, long_ma = calculate_moving_averages(stock_data)
    buy_signal = check_buy_signal(short_ma, long_ma)
    sell_signal = check_sell_signal(short_ma, long_ma)
    execute_trade(buy_signal, sell_signal)

schedule.every(1).minutes.do(main)  # Run main function every 1 minute

while True:
    schedule.run_pending()
    time.sleep(1)